# 2. Preprocessing:

We first repeat the import steps to get all the df and update the schema

In [16]:
import pandas as pd
from sodapy import Socrata
import numpy as np

In [18]:
# Import df
END_POINT = "data.cityofnewyork.us"
NEW_APP_TOKEN = 'u9SAL7T0QJyF6aY8OsLyS7gBX'
USERNAME = 'b.gutu@columbia.edu'
PASSWORD = 'M#-wMw6;]&]kU@r'



client = Socrata(END_POINT,
                 NEW_APP_TOKEN,
                 username=USERNAME,
                 password=PASSWORD)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("7x5e-2fxh",limit=20000) #feel free to unlimit this if you'd like. 2000 is sufficient for the hackathon (if not doing ML)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)


ConnectionError: HTTPSConnectionPool(host='df.cityofnewyork.us', port=443): Max retries exceeded with url: /resource/7x5e-2fxh.json?%24limit=20000 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000026700652AF0>: Failed to resolve 'df.cityofnewyork.us' ([Errno 11001] getaddrinfo failed)"))

Update the schema

In [14]:
# Adjusting df types
# print(list(df.dtypes))

types = {'numeric':0,'text':0,'floating_timestamp':0}
from schema import schema_dict

for column, dtype in schema_dict.items():
    if column not in df.columns:
        print('col not present',column)
        continue
    if dtype == 'number':
        types['numeric'] +=1
        df[column] = df[column].astype(float)  # or .astype(int) depending on the specifics
    elif dtype == 'text':
        types['text'] +=1

        df[column] = df[column].astype(str)
    elif dtype == 'floating_timestamp':
        types['floating_timestamp'] +=1

        df[column] = pd.to_datetime(df[column])

types


col not present property_id
col not present property_name
col not present parent_property_id
col not present parent_property_name
col not present year_ending
col not present nyc_borough_block_and_lot
col not present nyc_building_identification
col not present address_1
col not present city
col not present postal_code
col not present primary_property_type_self
col not present primary_property_type
col not present national_median_reference
col not present list_of_all_property_use
col not present largest_property_use_type
col not present largest_property_use_type_1
col not present _2nd_largest_property_use
col not present _2nd_largest_property_use_1
col not present _3rd_largest_property_use
col not present _3rd_largest_property_use_1
col not present year_built
col not present construction_status
col not present number_of_buildings
col not present occupancy
col not present metered_areas_energy
col not present metered_areas_water
col not present energy_star_score
col not present national_me

{'numeric': 0, 'text': 0, 'floating_timestamp': 0}

# Focusing on multifamily

In [ ]:
# Drop all columns where multifam is na

